In [5]:
import os
import shutil
import pandas as pd
import sys
# import pytest

# os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

In [3]:
!pip install pytest

In [6]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))



from experiment.benchmark_name import BenchmarkName
from experiment.flow_mapping import apply_flow_mapping, get_observation_harness_to_observed_source_map, get_observation_harness_to_string_set_map, get_observed_string_to_original_source_map
from intercept.decoded_apk_model import DecodedApkModel
from intercept.instrument import HarnessObservations
from tests.sample_results import get_mock_invocation_register_context

In [49]:
df1 = pd.DataFrame({"f": list(range(8)), 
                    "taint_function_name": ["taint1", "taint1", "taint1", "taint2", "taint2", "taint3", "taintmax", "taint_not_in_mapping"], 
                    "context_method": ["a()", "a()", "a()", "a()", "a()", "mlem()", "mlem()", "mlem()"],
                    "context_class": ["Foo", "Foo", "Foo", "Foo", "Foo", "Cats", "Cats", "Cats"],
                    "sink_function": ["sink1()", "sink2()", "sink1()", "sink2()", "sink1()", "sink2()", "sinkmax()", "sink_not_mapped()"],
                    "sink_context": ["Bar/zap()", "Bar/zap()", "Bar/zap()", "Bar/zap()", "Bar/zap()", "Bar/zap()", "Bar/zapmax()", "Bar/zapmax()"],
                    })
df1

,f,taint_function_name,context_method,context_class,sink_function,sink_context
0,0,taint1,a(),Foo,sink1(),Bar/zap()
1,1,taint1,a(),Foo,sink2(),Bar/zap()
2,2,taint1,a(),Foo,sink1(),Bar/zap()
3,3,taint2,a(),Foo,sink2(),Bar/zap()
4,4,taint2,a(),Foo,sink1(),Bar/zap()
5,5,taint3,mlem(),Cats,sink2(),Bar/zap()
6,6,taintmax,mlem(),Cats,sinkmax(),Bar/zapmax()
7,7,taint_not_in_mapping,mlem(),Cats,sink_not_mapped(),Bar/zapmax()


In [50]:
df2 = pd.DataFrame({"d": list(range(6)), 
                    "string_observed": ["secret1", "secret2", "***secret1***", "secretmax1", "secretmax2", "secret_unused"], 
                    "taint_function_name": ["taint1", "taint2", "taint3", "taintmax", "taintmax", "taint4"],
                    "context_method": ["a()", "a()", "mlem()", "mlem()", "mlem()", "mlem()"],
                    "context_class": ["Foo", "Foo", "Cats", "Cats", "Cats", "Cats"],
                    })
df2


,d,string_observed,taint_function_name,context_method,context_class
0,0,secret1,taint1,a(),Foo
1,1,secret2,taint2,a(),Foo
2,2,***secret1***,taint3,mlem(),Cats
3,3,secretmax1,taintmax,mlem(),Cats
4,4,secretmax2,taintmax,mlem(),Cats
5,5,secret_unused,taint4,mlem(),Cats


In [56]:
# def test_apply_flow_mapping_cross_demo():

df3 = pd.DataFrame({"a": list(range(6)), 
                    "expected_string_observed": ["secret1", "secret1", "secret2", "secret3", "***secret1***", "***secret2***"],
                    "scenario": ["profile", "profile-observation", "profile", "profile", "intercept", "intercept"],
                    "source_function": ["getSecret()", "getSecret()", "", "", "getImei()", "getLocale()"],
                    "source_function_context": ["", "CatTown/kibble()", "", "", "CatTown/build()", "CatTown/build()"],
                    })

df3

,a,expected_string_observed,scenario,source_function,source_function_context
0,0,secret1,profile,getSecret(),
1,1,secret1,profile-observation,getSecret(),CatTown/kibble()
2,2,secret2,profile,,
3,3,secret3,profile,,
4,4,***secret1***,intercept,getImei(),CatTown/build()
5,5,***secret2***,intercept,getLocale(),CatTown/build()


In [58]:
df2.merge(df3, how="left", left_on=["string_observed"], right_on=["expected_string_observed"])
# df2.merge(df3, how="outer", left_on=["string_observed", ], right_on=["expected_string_observed"])

,d,string_observed,taint_function_name,context_method,context_class,a,expected_string_observed,scenario,source_function,source_function_context
0,0,secret1,taint1,a(),Foo,0.0,secret1,profile,getSecret(),
1,0,secret1,taint1,a(),Foo,1.0,secret1,profile-observation,getSecret(),CatTown/kibble()
2,1,secret2,taint2,a(),Foo,2.0,secret2,profile,,
3,2,***secret1***,taint3,mlem(),Cats,4.0,***secret1***,intercept,getImei(),CatTown/build()
4,3,secretmax1,taintmax,mlem(),Cats,NaN,NaN,NaN,NaN,NaN
5,4,secretmax2,taintmax,mlem(),Cats,NaN,NaN,NaN,NaN,NaN
6,5,secret_unused,taint4,mlem(),Cats,NaN,NaN,NaN,NaN,NaN
